In [1]:
import pandas as pd
import json
import numpy as np

import string
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import pymorphy2
from pymystem3 import Mystem
from stop_words import get_stop_words
from keybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

from tqdm import tqdm

data_path = '../data/'

In [2]:
df = pd.read_csv(data_path + 'prepared_reviews_v3.csv')

In [3]:
df

,index,id,title,text,dateCreate,product,source
0,0,992651,Не выполняют условия акции!,В апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,sravni
1,1,998360,Жалоба на услугу «Газпром Бонус Премиум»,Купил услугу Газпром Бонус «Премиум» за 2 990 ...,2025-09-15T09:38:13.34818Z,debitCards,sravni
2,2,993744,Банк не отвечает за слова своих сотрудников! Н...,"Хочу поделиться историей, которая убила моё до...",2025-09-02T23:21:16.507166Z,debitCards,sravni
3,3,999494,Не выполняют условия акции!,В июне 2025 года я порекомендовал премиальную ...,2025-09-18T08:10:19.954986Z,debitCards,sravni
4,4,992291,Странный банк,Добрый день! В связи с устройством на новую ра...,2025-08-28T19:29:51.696341Z,debitCards,sravni
...,...,...,...,...,...,...,...
50352,50352,10885124,Под видом кредита 12.9% пытаются впарить креди...,Газпромбанк тут заманивал кредитом одобренным ...,2023-04-15 23:33:45,individual,banki
50353,50353,10872392,Неполное изменение номера в офисе/ отделении,Ситуация довольно странная для входа в интерне...,2023-03-30 12:39:01,individual,banki
50354,50354,10852603,Один из отличных банков,В банке нравится все. И обслуживание на отличн...,2023-03-10 15:04:12,individual,banki
50355,50355,10852597,Обращение в контакт центр 10 баллов,"Обращение в контакт центр 10 баллов, а вот раб...",2023-03-10 14:57:02,individual,banki


In [4]:
df["review"] = (
    df["title"].fillna('') + " " + df["text"].fillna('')
).str.replace("\xa0", " ", regex=False).str.replace("\n", " ", regex=False)

In [5]:
df

,index,id,title,text,dateCreate,product,source,review
0,0,992651,Не выполняют условия акции!,В апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,sravni,Не выполняют условия акции! В апреле 2025 года...
1,1,998360,Жалоба на услугу «Газпром Бонус Премиум»,Купил услугу Газпром Бонус «Премиум» за 2 990 ...,2025-09-15T09:38:13.34818Z,debitCards,sravni,Жалоба на услугу «Газпром Бонус Премиум» Купил...
2,2,993744,Банк не отвечает за слова своих сотрудников! Н...,"Хочу поделиться историей, которая убила моё до...",2025-09-02T23:21:16.507166Z,debitCards,sravni,Банк не отвечает за слова своих сотрудников! Н...
3,3,999494,Не выполняют условия акции!,В июне 2025 года я порекомендовал премиальную ...,2025-09-18T08:10:19.954986Z,debitCards,sravni,Не выполняют условия акции! В июне 2025 года я...
4,4,992291,Странный банк,Добрый день! В связи с устройством на новую ра...,2025-08-28T19:29:51.696341Z,debitCards,sravni,Странный банк Добрый день! В связи с устройств...
...,...,...,...,...,...,...,...,...
50352,50352,10885124,Под видом кредита 12.9% пытаются впарить креди...,Газпромбанк тут заманивал кредитом одобренным ...,2023-04-15 23:33:45,individual,banki,Под видом кредита 12.9% пытаются впарить креди...
50353,50353,10872392,Неполное изменение номера в офисе/ отделении,Ситуация довольно странная для входа в интерне...,2023-03-30 12:39:01,individual,banki,Неполное изменение номера в офисе/ отделении С...
50354,50354,10852603,Один из отличных банков,В банке нравится все. И обслуживание на отличн...,2023-03-10 15:04:12,individual,banki,Один из отличных банков В банке нравится все. ...
50355,50355,10852597,Обращение в контакт центр 10 баллов,"Обращение в контакт центр 10 баллов, а вот раб...",2023-03-10 14:57:02,individual,banki,Обращение в контакт центр 10 баллов Обращение ...


# LDA Model

In [6]:
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [21]:
# ---------------------------
# 1) Параметры и данные
# ---------------------------
df = df.loc[:1000].copy()
df['content'] = df['title'].fillna('') + '. ' + df['text'].fillna('')
docs_raw = df['content'].astype(str).tolist()
n_docs = len(docs_raw)
print(f"Документов: {n_docs}")

Документов: 1001


In [22]:
import nltk
try:
    _ = nltk.corpus.stopwords.words('russian')
except LookupError:
    nltk.download('stopwords')
from nltk.corpus import stopwords
RU_STOPWORDS = set(stopwords.words('russian'))

# добавить свои стоп-слова (при желании)
custom_stopwords = {
    'фраза_пример', 'sravni', 'газпром'  # удалите/дополните по необходимости
}
RU_STOPWORDS |= set(custom_stopwords)

In [23]:
import spacy
nlp = spacy.load("ru_core_news_sm", disable=["ner", "parser"])

def lemmatize_text(text):
    doc = nlp(text)
    return [tok.lemma_.lower() for tok in doc if tok.is_alpha and not tok.is_stop]

# пример
text = "В апреле 2025 года я рекомендовала дебетовую карту."
tokens = lemmatize_text(text)
print(tokens)
# ['апрель', 'год', 'рекомендовать', 'дебетовый', 'карта']

INFO:Loading dictionaries from /home/gleb/LCT_2025/.venv/lib/python3.13/site-packages/pymorphy3_dicts_ru/data
INFO:format: 2.4, revision: 417150, updated: 2022-01-08T22:09:24.565962


['апрель', 'год', 'рекомендовать', 'дебетовый', 'карта']


In [24]:
def preprocess_text_spacy(text, min_word_len=2, stopwords_set=RU_STOPWORDS):
    doc = nlp(text.lower())
    tokens = [tok.lemma_ for tok in doc if tok.is_alpha and len(tok.lemma_)>=min_word_len and tok.lemma_ not in stopwords_set]
    return tokens


In [25]:
# ---------------------------
# 4) Токенизация и очистка
# ---------------------------
TOKEN_RE = re.compile(r'[А-Яа-яёЁA-Za-z0-9\-]+', re.UNICODE)

def preprocess_text(text,
                    min_word_len=2,
                    stopwords_set=RU_STOPWORDS,
                    do_lemmatize=True):
    # lower
    text = str(text).lower()
    # убрать ссылки, e-mail, html-теги, лишние символы
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'\S+@\S+', ' ', text)
    text = re.sub(r'<[^>]+>', ' ', text)
    # заменить дефисы и знаки
    tokens = TOKEN_RE.findall(text)
    tokens = [t for t in tokens if len(t) >= min_word_len]
    # убрать числа если хотите (закомментируйте если важны)
    tokens = [t for t in tokens if not t.isdigit()]
    # убрать стоп-слова
    tokens = [t for t in tokens if t not in stopwords_set]
    if do_lemmatize:
        tokens = [lemmatize_token(t) for t in tokens]
    # окончательная фильтрация (повторно)
    tokens = [t for t in tokens if t not in stopwords_set and len(t) >= min_word_len]
    return tokens

# Применим предобработку (параллельно можно ускорять, здесь tqdm для прогресса)
texts = []
for doc in tqdm(docs_raw, desc="Preprocessing"):
    toks = preprocess_text_spacy(doc)
    texts.append(toks)

# Пример: вывести первые 5 обработанных документов
for i in range(min(5, n_docs)):
    print(i, texts[i][:30])


Preprocessing: 100%|██████████| 1001/1001 [00:22<00:00, 45.07it/s]

0 ['выполнять', 'условие', 'акция', 'апрель', 'год', 'рекомендовать', 'дебетовый', 'карта', 'друг', 'согласно', 'акция', 'банк', 'привести', 'друг', 'далее', 'май', 'мною', 'друг', 'условие', 'выполнить', 'вознаграждение', 'акция', 'прийти', 'друг', 'июль', 'прийти', 'начинать', 'июнь', 'писать', 'чат']
1 ['жалоба', 'услуга', 'бонус', 'премиум', 'купить', 'услуга', 'бонус', 'премиум', 'цель', 'абсолютно', 'конкретный', 'подключить', 'категория', 'спорт', 'путешествие', 'внимательно', 'выбрать', 'именно', 'спорт', 'учитывать', 'жена', 'ранее', 'возникать', 'аналогичный', 'проблема', 'несмотря', 'правильный', 'выбор', 'итог', 'программа']
2 ['банк', 'отвечать', 'слово', 'свой', 'сотрудник', 'невыплаченный', 'кэшбэк', 'игнорирование', 'обращение', 'хотеть', 'поделиться', 'история', 'которая', 'убить', 'моё', 'доверие', 'газпромбанк', 'участвовать', 'акция', 'кешбэк', 'ежедневный', 'трата', 'март', 'год', 'специально', 'звонить', 'горячий', 'линия', 'уточнить', 'фиксироваться']
3 ['выполня

In [26]:
# ---------------------------
# 5) N-grams (опционально) — биграммы/триграммы
# ---------------------------
from gensim.models import Phrases, phrases

# если корпус небольшой, уменьшаем порог
min_count_phrases = 5 if n_docs > 200 else 2
phrases_bigram = Phrases(texts, min_count=min_count_phrases, threshold=10)
bigram = phrases.Phraser(phrases_bigram)
texts_bi = [bigram[t] for t in texts]

# также можно добавить триграммы:
phrases_trigram = Phrases(texts_bi, min_count=min_count_phrases, threshold=15)
trigram = phrases.Phraser(phrases_trigram)
texts_final = [trigram[bigram[t]] for t in texts]  # содержит биграммы и триграммы

ModuleNotFoundError: No module named 'gensim'

In [ ]:

# ---------------------------
# 6) Словарь и корпус (gensim)
# ---------------------------
from gensim.corpora import Dictionary
dictionary = Dictionary(texts_final)

# адаптивные фильтры: для маленьких датасетов значения должны быть маленькими
no_below = max(2, int(0.001 * n_docs))   # min doc freq
no_above = 0.6                            # max document proportion
keep_n = 1001#50000

dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=keep_n)
dictionary.compactify()
print(f"Размер словаря после фильтрации: {len(dictionary)}")

# корпус — bag-of-words
corpus = [dictionary.doc2bow(text) for text in texts_final]

# если словарь слишком маленький (например на 3 документах), ослабим фильтр
if len(dictionary) < 10:
    dictionary = Dictionary(texts)  # попробовать без n-grams
    dictionary.filter_extremes(no_below=1, no_above=0.9, keep_n=50000)
    corpus = [dictionary.doc2bow(t) for t in texts]
    print("Пересоздан словарь более мягкими параметрами, размер:", len(dictionary))